# Notebook to experiment with testing:

## Code:

In [6]:
import numpy as np
import random
import torch
from gymnasium import spaces

SEED = 42
# Python RNG
random.seed(SEED)

# NumPy RNG
np.random.seed(SEED)

# PyTorch RNG (CPU + GPU)
torch.manual_seed(SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed(SEED)
    torch.cuda.manual_seed_all(SEED)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False


In [7]:
import sys
sys.path.append("/home/martina/codi2/4year/tfg")  # add parent folder of general.py

from general import prepare, Glioblastoma, GlioblastomaPositionalEncoding, testing
from training_dqn import DQNPaper, DQNPositionalEncoding, DQNMriLite
from training_agents import DQNAgent
from training_buffers import ReplayBuffer

In [8]:
test_pairs = prepare(mode='test')

Preparing testing set.
✅ Found 100 pairs out of 100 listed in CSV.


# TESTING:

In [16]:
# load model to test:
LR = 5e-5

CURRENT_CONFIG = {
    'grid_size': 6,
    'rewards': [3.0, -1.0, -0.2],  # [on_tumor, off_tumor, move]
    'action_space': spaces.Discrete(5), 
    'max_steps': 40, 
    'stop': False
}

model_name = 'Extension048_best'
env = GlioblastomaPositionalEncoding(*test_pairs[0], **CURRENT_CONFIG)

model = DQNPositionalEncoding(env, learning_rate=LR, device='cpu')
model.load_state_dict(torch.load(f"{model_name}.dat"))

agent = DQNAgent(env_config=CURRENT_CONFIG, dnnetwork=model, buffer_class=ReplayBuffer, train_pairs=test_pairs,
                 env_class=GlioblastomaPositionalEncoding,
                 epsilon=0.00)  # very low epsilon for testing

In [17]:
overall_results = testing(agent, test_pairs, agent_type="dqn", num_episodes=len(test_pairs), env_config=CURRENT_CONFIG, save_gifs=True, gif_folder=f"GIFs_Testing_{model_name}")

Saved GIF for episode 0 at GIFs_Testing_Extension048_best/episode_0_002_58.gif
Saved GIF for episode 10 at GIFs_Testing_Extension048_best/episode_10_013_86.gif
Saved GIF for episode 20 at GIFs_Testing_Extension048_best/episode_20_024_49.gif
Saved GIF for episode 30 at GIFs_Testing_Extension048_best/episode_30_038_84.gif
Saved GIF for episode 40 at GIFs_Testing_Extension048_best/episode_40_052_98.gif
Saved GIF for episode 50 at GIFs_Testing_Extension048_best/episode_50_104_74.gif
Saved GIF for episode 60 at GIFs_Testing_Extension048_best/episode_60_176_99.gif
Saved GIF for episode 70 at GIFs_Testing_Extension048_best/episode_70_204_52.gif
Saved GIF for episode 80 at GIFs_Testing_Extension048_best/episode_80_260_62.gif
Saved GIF for episode 90 at GIFs_Testing_Extension048_best/episode_90_300_107.gif

TEST RESULTS (DQN Agent)
Success Rate: 52.00%
Average Episode Reward: 18.24
Average Steps to Find Tumor: 8.29
Average Tumor Rewards per Episode: 6.95
Tumor Size Statistics:
  Biggest Tumor: 